# Installing Net2Brain

In [ ]:
#use conda env net2brain

In [ ]:
#!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

# Step 1: Feature Extraction

## Using `FeatureExtractor` with a model from Net2Brain

The FeatureExtractor class provides an interface for extracting features from a given model. When initializing this class, you can customize its behavior by setting various parameters:

- `model` (required): The model from which you want to extract features. Either string in combination with a netset (next parameter), or a variable with a model-type.
- `netset` (optional): The netset (collection of networks) that the model belongs to.
- `layers_to_extract` (optional): A list of layer names or indices from which you want to extract features. Default is None, indicating that all layers preset in the toolbox will be used.
- `device` (optional): The device on which to perform the computations, e.g., 'cuda' for GPU or 'cpu' for CPU. Default is None, which will use the device specified in the global PyTorch settings.
- `transforms` (optional): A list of data preprocessing transforms to apply to the input data before passing it through the model. Default is None, which uses the preset transformations
- `pretrained` (optional): A boolean flag indicating whether to use a pretrained model (if available) or to initialize the model with random weights. Default is True, which means that a pretrained model will be used if possible.

- - -


First we need to a dataset to play around with. For that we will use the dataset by [Micheal F. Bonner (2017)](https://www.pnas.org/doi/full/10.1073/pnas.1618228114), which we can download using the `load_dataset` function

In [ ]:
# from net2brain.utils.download_datasets import load_dataset
# stimuli_path, roi_path = load_dataset("bonner_pnas2017")

### Initating FeatureExtractor


To extract the activations of a pretrained model from a netset, you can use the FeatureExtractor class. First, you need to initialize the class by providing the name of the model and the name of the netset. You can find a suitable model and netset by exploring the taxonomy options available in the Net2Brain toolbox, as shown in the previous notebook "0_Exploring_Net2Brain". For instance, in the following example, we will use AlexNet from the standard netset.

In [10]:
from net2brain.feature_extraction import FeatureExtractor
fx = FeatureExtractor(model='ResNet50', netset='Standard', device='cpu') #cuda not working now, try dif gpu?

The `extract` method computes feature extraction from an image dataset. It takes in the following parameters:

- `data_path` (required): The path to the images from which to extract the features. The images must be in JPEG or PNG format.
- `save_path` (optional): The path to save the extracted features to. If None, the folder where the features are saved is named after the current date in the format "{year}{month}{day}{hour}{minute}".
- `layers_to_extract` (optional): A list of layer names or indices from which to extract features. If None, the specified layers will be used.
- `consolidate_per_layer` (optional): The features are extracted image-wise. This is defaulted to true and will consolidate them per layer if not set to False. Defautls to True.
- `dim_reduction` (optional): Type of dimensionality reduction (For now: SRP) for extracted features. Defaults to None.
- `n_components` (optonal): Number of components for dimensionality reduction. Defaults to 50.

In [12]:
from net2brain.feature_extraction import FeatureExtractor
import os

stim='DM'
stimuli_path = f'../data/{stim}_frames/'
save_path = f'../data/{stim}_frames_resnet50/'

if not os.path.isdir(save_path):
    os.mkdir(save_path) 
    
#relu, maxpool, block1, block2, block3, block4, avgpool
layers_to_extract=['relu','maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool']

#fx = FeatureExtractor(model='AlexNet', netset='Standard', device='cpu')
fx.extract(data_path=stimuli_path, save_path=save_path, layers_to_extract=layers_to_extract)  #, consolidate_per_layer=False)

100%|██████████| 750/750 [01:53<00:00,  6.64it/s]


Consolidating data per layer...


100%|██████████| 750/750 [00:28<00:00, 26.67it/s]


__Net2Brain__ chooses by default from which layers of the model to extract the features from. You can inspect which layers are selected by default by calling the `layers_to_extract` attribute:

In [2]:
fx.layers_to_extract

['layer1', 'layer2', 'layer3', 'layer4']

These are not all the layers that **can** be extracted. If you want to see all the layers that can possibly be extracted you you call `get_all_layers()`.

In [3]:
fx.get_all_layers()

['',
 'conv1',
 'bn1',
 'relu',
 'maxpool',
 'layer1',
 'layer1.0',
 'layer1.0.conv1',
 'layer1.0.bn1',
 'layer1.0.conv2',
 'layer1.0.bn2',
 'layer1.0.conv3',
 'layer1.0.bn3',
 'layer1.0.relu',
 'layer1.0.downsample',
 'layer1.0.downsample.0',
 'layer1.0.downsample.1',
 'layer1.1',
 'layer1.1.conv1',
 'layer1.1.bn1',
 'layer1.1.conv2',
 'layer1.1.bn2',
 'layer1.1.conv3',
 'layer1.1.bn3',
 'layer1.1.relu',
 'layer1.2',
 'layer1.2.conv1',
 'layer1.2.bn1',
 'layer1.2.conv2',
 'layer1.2.bn2',
 'layer1.2.conv3',
 'layer1.2.bn3',
 'layer1.2.relu',
 'layer2',
 'layer2.0',
 'layer2.0.conv1',
 'layer2.0.bn1',
 'layer2.0.conv2',
 'layer2.0.bn2',
 'layer2.0.conv3',
 'layer2.0.bn3',
 'layer2.0.relu',
 'layer2.0.downsample',
 'layer2.0.downsample.0',
 'layer2.0.downsample.1',
 'layer2.1',
 'layer2.1.conv1',
 'layer2.1.bn1',
 'layer2.1.conv2',
 'layer2.1.bn2',
 'layer2.1.conv3',
 'layer2.1.bn3',
 'layer2.1.relu',
 'layer2.2',
 'layer2.2.conv1',
 'layer2.2.bn1',
 'layer2.2.conv2',
 'layer2.2.bn2',
 '

If you wish to change the layers to be extracted you can add it to the `extract` function like with the parameter 
```
fx.extract(..., layers_to_extract=[your_layers])
```

## explore features

In [25]:
import os
import numpy as np
import glob

stim='DM'
save_path = f'../data/{stim}_frames_resnet50/'

#print the
emb_list = glob.glob(f'{save_path}*.npz')

layers_to_extract=['relu','maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool']

print('ResNet50 frame embeddings')
# for emb_f in emb_list:
#     emb = np.load(f'{emb_f}')
#     print(emb['frame_0000.jpg'].shape, os.path.splitext(os.path.basename(emb_f))[0])
for layer in layers_to_extract:
    emb = np.load(f'{save_path}{layer}.npz')
    print(emb['frame_0000.jpg'].shape, layer)


ResNet50 frame embeddings
(1, 64, 112, 112) relu
(1, 64, 56, 56) maxpool
(1, 256, 56, 56) layer1
(1, 512, 28, 28) layer2
(1, 1024, 14, 14) layer3
(1, 2048, 7, 7) layer4
(1, 2048, 1, 1) avgpool
